In [ ]:
# verigy whether nonflat/distort boundary models are easier to be adversarial attack
# time or distortion
# here we choose time

In [8]:
# fgsm attack
# import modules
from deepvisualinsight.MMS import MMS
from deepvisualinsight import utils
import sys
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import json
import tensorflow as tf
import torchvision.transforms as transforms
import torchvision

In [ ]:
def cw_l2_attack(model, image, label, device,c=1, kappa=0, max_iter=25, learning_rate=0.1, verbose=1):

    # Get loss2
    def f(x):
        outputs = model(x)
        one_hot_labels = torch.eye(len(outputs[0]))[label].to(device)

        i, _ = torch.max((1-one_hot_labels)*outputs, dim=1)
        j = torch.masked_select(outputs, one_hot_labels.byte())

        return torch.clamp(j-i, min=-kappa)

    successful = False

    # initialize w : the noise
    w = torch.zeros_like(image, requires_grad=True).to(device)
    optimizer = optim.Adam([w], lr=learning_rate) # an optimizer specifically for w

    for step in range(max_iter):
        # w is the noise added to the original image, restricted to be [-1, 1]
        a = image + torch.tanh(w)

        loss1 = nn.MSELoss(reduction='sum')(a, image) # ||x-x'||2
        loss2 = torch.sum(c*f(a)) # c*{f(label) - f(target_cls)}

        cost = loss1 + loss2

        # Backprop: jointly optimize the loss
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        # New prediction
        with torch.no_grad():
            pred_new = model(a)

        # Stop when ...
        # successfully flip the label
        if torch.argmax(pred_new, dim=1).item() != label.item():
            successful = True
            break

        if verbose > 0:
            print('- Learning Progress : %2.2f %% ' %((step+1)/max_iter*100), end='\r')

    # w is the noise added to the original image, restricted to be [-1, 1]
    # attack_images = image + torch.tanh(w)
    r = torch.tanh(w).detach.cpu().numpy()
    r = np.sqrt(np.mean(np.power(r, 2)))

    return r, successful, step


In [ ]:
criterion = nn.CrossEntropyLoss()
softmax = torch.nn.Softmax(dim=-1)
epsilons = [.01,.03,.05,.1]
epsilon = .01

def fgsm_attack(image, epsilon, data_grad):

    # Collect the element-wise sign of the data gradient
    sign_data_grad = torch.sign(data_grad)

    perturbed_image = image + epsilon*sign_data_grad

    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)

    return perturbed_image

In [2]:
content_path = "E:\DVI_exp_data\\active_learning\\same_start\\random"
sys.path.append(content_path)

from Model.model import *
net = ResNet18()
classes = ("airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

mms = MMS(content_path, net, 1, 10, 1, 512, 10, classes, cmap="tab10", resolution=100, neurons=256,
          verbose=1, temporal=False, split=-1, advance_border_gen=True, attack_device="cuda:0")

Finish loading content!


In [3]:
# hyperparameters
EPOCH=180

In [4]:
model_location = os.path.join(mms.model_path, "Epoch_{:d}".format(EPOCH), "subject_model.pth")
mms.model.load_state_dict(torch.load(model_location, map_location=torch.device("cpu")))
mms.model = mms.model.to(mms.device)
mms.model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [7]:
# fgsm: how many adv samples we can get
# ifgsm: set a time limit
TEST_BUDGET = 10000

CIFAR_NORM = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(*CIFAR_NORM)])
testset = torchvision.datasets.CIFAR10(root='data', train=False,
                                       download=True, transform=transform)
test_subset = torch.utils.data.Subset(testset, np.arange(TEST_BUDGET))
testloader = torch.utils.data.DataLoader(test_subset, batch_size=1,
                                         shuffle=False, num_workers=2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("attack with epsilon {}...".format(epsilon))

Files already downloaded and verified
attack with epsilon 0.01...


4804

In [ ]:
succ = 0
for i, (data, target) in enumerate(testloader, 0):
    # Send the data and label to the device
    data, target = data.to(device), target.to(device)
    # Set requires_grad attribute of tensor. Important for Attack
    data.requires_grad = True
    # Forward pass the data through the model
    output = mms.model(data)
    init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

    # If the initial prediction is wrong, dont bother attacking, just move on
    if init_pred.item() != target.item():
        continue

    # Calculate the loss
    loss = torch.nn.functional.nll_loss(output, target)

    # Zero all existing gradients
    mms.model.zero_grad()

    # Calculate gradients of model in backward pass
    loss.backward()

    # Collect datagrad
    data_grad = data.grad.data

    # Call FGSM Attack
    perturbed_data = fgsm_attack(data, epsilon, data_grad)

    # Re-classify the perturbed image
    output = mms.model(perturbed_data)

    # Check for success
    final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
    if final_pred.item() != target.item():
        succ += 1
succ


In [ ]:
TEST_BUDGET = 10000
succ = 0
r = 0
wrong_pred = 0
for i, (data, target) in enumerate(testloader, 0):
    data, target = data.to(device), target.to(device)
    output = mms.model(data)
    init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
    # If the initial prediction is wrong, dont bother attacking, just move on
    if init_pred.item() != target.item():
        wrong_pred += 1
        continue
    r_, s,_ = cw_l2_attack(mms.model, data, target, device=mms.device, targeted=False, c=0.1)
    if s:
        succ += 1
        r += r_
succ,TEST_BUDGET-wrong_pred, r/succ

In [11]:
sys.path.remove(content_path)
sys.path

['E:\\xianglin\\git_space\\DeepVisualInsight',
 'd:\\downloads\\anaconda\\envs\\dr2\\python36.zip',
 'd:\\downloads\\anaconda\\envs\\dr2\\DLLs',
 'd:\\downloads\\anaconda\\envs\\dr2\\lib',
 'd:\\downloads\\anaconda\\envs\\dr2',
 '',
 'C:\\Users\\xianglin\\AppData\\Roaming\\Python\\Python36\\site-packages',
 'd:\\downloads\\anaconda\\envs\\dr2\\lib\\site-packages',
 'e:\\xianglin\\git_space\\parametricumap_paper',
 'd:\\downloads\\anaconda\\envs\\dr2\\lib\\site-packages\\win32',
 'd:\\downloads\\anaconda\\envs\\dr2\\lib\\site-packages\\win32\\lib',
 'd:\\downloads\\anaconda\\envs\\dr2\\lib\\site-packages\\Pythonwin',
 'd:\\downloads\\anaconda\\envs\\dr2\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\xianglin\\.ipython',
 'E:\\DVI_exp_data\\active_learning\\LeastConfidence',
 'F:\\xianglin\\PyCharm 2021.1.1\\plugins\\python\\helpers\\pydev',
 'F:\\xianglin\\PyCharm 2021.1.1\\plugins\\python\\helpers-pro\\jupyter_debug']

In [9]:
content_path = "E:\DVI_exp_data\\active_learning\\same_start\\LL4AL"
sys.path.append(content_path)

from Model.model import *
net = ResNet18()
classes = ("airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

mms = MMS(content_path, net, 1, 10, 1, 512, 10, classes, cmap="tab10", resolution=100, neurons=256,
          verbose=1, temporal=False, split=-1, advance_border_gen=True, attack_device="cuda:0")

Finish loading content!


In [10]:
# hyperparameters
EPOCH=180

In [11]:
model_location = os.path.join(mms.model_path, "Epoch_{:d}".format(EPOCH), "subject_model.pth")
state_dict = torch.load(model_location)
mms.model.load_state_dict(state_dict)
mms.model.to(mms.device)
mms.model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [13]:
# fgsm: how many adv samples we can get
# ifgsm: set a time limit
TEST_BUDGET = 10000

CIFAR_NORM = ((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(*CIFAR_NORM)])
testset = torchvision.datasets.CIFAR10(root='data', train=False,
                                       download=True, transform=transform)
test_subset = torch.utils.data.Subset(testset, np.arange(TEST_BUDGET))
testloader = torch.utils.data.DataLoader(test_subset, batch_size=1,
                                         shuffle=False, num_workers=2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print("attack with epsilon {}...".format(epsilon))

succ = 0
for i, (data, target) in enumerate(testloader, 0):
    # Send the data and label to the device
    data, target = data.to(device), target.to(device)
    # Set requires_grad attribute of tensor. Important for Attack
    data.requires_grad = True
    # Forward pass the data through the model
    output = mms.model(data)
    init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability

    # If the initial prediction is wrong, dont bother attacking, just move on
    if init_pred.item() != target.item():
        continue

    # Calculate the loss
    loss = torch.nn.functional.nll_loss(output, target)

    # Zero all existing gradients
    mms.model.zero_grad()

    # Calculate gradients of model in backward pass
    loss.backward()

    # Collect datagrad
    data_grad = data.grad.data

    # Call FGSM Attack
    perturbed_data = fgsm_attack(data, epsilon, data_grad)

    # Re-classify the perturbed image
    output = mms.model(perturbed_data)

    # Check for success
    final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
    if final_pred.item() != target.item():
        succ += 1
succ

Files already downloaded and verified
attack with epsilon 0.01...


4958

In [ ]:
TEST_BUDGET = 10000
succ = 0
r = 0
wrong_pred = 0
for i, (data, target) in enumerate(testloader, 0):
    data, target = data.to(device), target.to(device)
    output = mms.model(data)
    init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
    # If the initial prediction is wrong, dont bother attacking, just move on
    if init_pred.item() != target.item():
        wrong_pred += 1
        continue
    r_, s,_ = cw_l2_attack(mms.model, data, target, device=mms.device, targeted=False, c=0.1)
    if s:
        succ += 1
        r += r_
succ,TEST_BUDGET-wrong_pred, r/succ